### Kreiranje pogosto skupaj kupljenih parov izdelkov in testne množice

<br>

### 1. Uvoz in filtriranje podatkov

In [1]:
# Uvoz knjižnic
import pandas as pd
import numpy as np

from itertools import combinations, groupby
from collections import Counter

In [2]:
orders = pd.read_csv('../data/orders.csv')[['order_id', 'user_id', 'eval_set', 'order_number']]
orders.head()

,order_id,user_id,eval_set,order_number
0,2539329,1,prior,1
1,2398795,1,prior,2
2,473747,1,prior,3
3,2254736,1,prior,4
4,431534,1,prior,5


In [3]:
order_products_prior = pd.read_csv('../data/order_products__prior.csv')[['order_id', 'product_id']]
order_products_prior.head()

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035


In [4]:
order_products_train = pd.read_csv('../data/order_products__train.csv')[['order_id', 'product_id']]
order_products_train.head()

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


In [5]:
len(order_products_prior) + len(order_products_train)

33819106

In [6]:
# Ponekod zadnje naročilo pripada množici `test`, ki za našo implementacijo ni relavantna,
# saj je ta del tekmovanja Kaggla
orders = orders[orders['eval_set'] != 'test']

In [7]:
# Vsa zadnja naročila, kjer je ponekod potrebno oznako `prior` spremeniti na `train`
# Istočasno je potrebno ta naročila prestaviti iz tabele `order_products_prior` v tabelo `order_products_train`
last_orders = orders.groupby('user_id').tail(1)

In [8]:
orders['eval_set'] = orders['order_id'].isin(last_orders['order_id']).replace({True: 'train', False: 'prior'})

In [9]:
# Omenjena naročila (spodaj) je potrebno prenesti
orders_to_move = last_orders[last_orders['eval_set'] == 'prior']['order_id']
orders_to_move.head()

37     1402502
43     2557754
52      998866
95     1468214
101     221248
Name: order_id, dtype: int64

In [10]:
df_to_move = order_products_prior[order_products_prior['order_id'].isin(orders_to_move)]
df_to_move.head()

,order_id,product_id
404,51,30274
405,51,3594
406,51,14994
407,51,45433
408,51,44514


In [11]:
# Zgornjo tabelo dodamo tabeli `order_products_train`...
order_products_train = order_products_train.append(df_to_move, ignore_index=True)

# ... in jo odstranimo iz tabele `order_products_prior`
order_products_prior = order_products_prior[~order_products_prior['order_id'].isin(orders_to_move)]

# Sprostitev pomnilnika
del orders_to_move, df_to_move

# Vsota velikosti obeh tabel je enaka začetni
len(order_products_prior) + len(order_products_train)

33819106

<br>

### 2. Delitev podatkov na učno in testno množico

In [12]:
# 90 % uporabnikov predstavlja učna množica
# 10 % uporabnikov predstavlja testna množica
users = orders['user_id'].unique()
users_mask = np.random.rand(len(users)) > 0.1
orders_train = orders[orders['user_id'].isin(users[users_mask])]
orders_test = orders[orders['user_id'].isin(users[~users_mask])]

In [13]:
del orders, users, users_mask

In [14]:
len(orders_train) / len(orders_test)

8.978299377340935

In [15]:
orders_train.head()

,order_id,user_id,eval_set,order_number
0,2539329,1,prior,1
1,2398795,1,prior,2
2,473747,1,prior,3
3,2254736,1,prior,4
4,431534,1,prior,5


In [ ]:
# Shranjevanje testne množice izdelkov v datoteko orders_test.csv, saj je pare izdelkov bilo
# potrebno implementirati v 2 datotekah, sicer je sledila napaka `MemoryError`
orders_test.to_csv('../data/orders_test.csv', index=False)

<br>

### 3. Najpogostejši pari izdelkov

In [16]:
# Potrebujemo pare: naročilo - izdelek
orders_to_process = orders_train.merge(order_products_prior.merge(order_products_train, how='outer'))[['order_id', 'product_id']] # 28 480 438
orders_to_process.head()

,order_id,product_id
0,2539329,196
1,2539329,14084
2,2539329,12427
3,2539329,26088
4,2539329,26405


In [17]:
# Pretvorba zgorje tabele v matriko števil
orders_matrix = orders_to_process.values
orders_matrix

array([[2539329,     196],
       [2539329,   14084],
       [2539329,   12427],
       ...,
       [ 272231,   42606],
       [ 272231,   37966],
       [ 272231,   39216]], dtype=int64)

In [18]:
# Funkcija, ki vrne vse možne pare izdelkov, podanih v obliki matrike
def product_pairs(orders_matrix):
    for order_id, order in groupby(orders_matrix, lambda x: x[0]):
        # Vsi izdelki določenega naročila
        products = [product[1] for product in order]
        
        # Generiranje vseh parov izdelkov
        pairs = combinations(products, 2)
        for pair in pairs:
            yield pair

In [19]:
# Pari produktov 
pairs = product_pairs(orders_matrix)

In [20]:
# Števec parov produktov
pairs_count = Counter(pairs)

In [21]:
pairs_count

Counter({(196, 14084): 56,
         (196, 12427): 502,
         (196, 26088): 90,
         (196, 26405): 47,
         (14084, 12427): 8,
         (14084, 26088): 13,
         (14084, 26405): 2,
         (12427, 26088): 42,
         (12427, 26405): 15,
         (26088, 26405): 3,
         (196, 10258): 210,
         (196, 13176): 1379,
         (196, 13032): 294,
         (10258, 12427): 57,
         (10258, 13176): 90,
         (10258, 26088): 13,
         (10258, 13032): 26,
         (12427, 13176): 374,
         (12427, 13032): 67,
         (13176, 26088): 286,
         (13176, 13032): 314,
         (26088, 13032): 18,
         (196, 25133): 38,
         (196, 30450): 226,
         (12427, 10258): 52,
         (12427, 25133): 18,
         (12427, 30450): 80,
         (10258, 25133): 6,
         (10258, 30450): 42,
         (25133, 30450): 50,
         (10258, 26405): 3,
         (25133, 26405): 3,
         (196, 10326): 120,
         (196, 17122): 402,
         (196, 41787): 183,
   

In [22]:
# Tabela, ki prikazuje pare izdelkov, skupaj s frekvenco skupne pojavitve
pairs = pd.Series(pairs_count).to_frame('count').reset_index()
pairs.columns = ['product_one', 'product_two', 'count']

In [35]:
del pairs_count

In [23]:
pairs.head()

,product_one,product_two,count
0,196,14084,56
1,196,12427,502
2,196,26088,90
3,196,26405,47
4,14084,12427,8


In [24]:
pairs[(pairs['product_one'] == 196) & (pairs['product_two'] == 14084)]

,product_one,product_two,count
0,196,14084,56


In [25]:
pairs[(pairs['product_one'] == 14084) & (pairs['product_two'] == 196)]

,product_one,product_two,count
733790,14084,196,40


#### 3.1 Popravek parov

In [33]:
# Obstaja možnost, da se par pojavi v dveh različnih vrstnih redih (primer zgoraj)
# To popravimo
pairs['product_one'] = pairs['product_one'].agg(lambda x: str(x))
pairs['product_two'] = pairs['product_two'].agg(lambda x: str(x))
pairs['combine'] = pairs['product_one'] + ' ' + pairs['product_two']
pairs.head()

,product_one,product_two,count,combine
0,196,14084,56,196 14084
1,196,12427,502,196 12427
2,196,26088,90,196 26088
3,196,26405,47,196 26405
4,14084,12427,8,14084 12427


In [1]:
pairs['combine'] = pairs['combine'].agg(lambda x: ' '.join(set(x.split(' '))))

In [2]:
pairs = pairs.groupby('combine').sum().reset_index()[['combine', 'count']]

In [4]:
pairs['product_one'] = pairs['combine'].agg(lambda x: x.split(' ')[0])
pairs['product_two'] = pairs['combine'].agg(lambda x: x.split(' ')[1])
pairs = pairs[['product_one', 'product_two', 'count']].astype('int64')

In [5]:
pairs = pairs.sort_values(by='count', ascending=False).reset_index(drop=True)

In [6]:
# Shranjevanje parov izdelkov v datoteko pairs.csv
pairs.to_csv('../data/pairs.csv', index=False)

<br>

### Priporočilni sistem z uporabo pogostosti parov izdelkov, ustvarjen v tej datoteki, je implementiran v datoteki `2_Priporočilni_sistem.ipynb`.